In [1]:
import pandas as pd
import numpy as np
#import keras
#from keras import backend as K

import tensorflow
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from numpy.random import seed
seed(101)
tensorflow.random.set_seed(101)

import os

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
os.listdir('D:/Анна/универ/ДИПЛОМ/data/ISIC')

['!code',
 'data backup',
 'data backup aug',
 'data backup noaug',
 'ISIC_2019_Test_Input',
 'ISIC_2019_Test_Metadata.csv',
 'ISIC_2019_Training_GroundTruth.csv',
 'ISIC_2019_Training_Input',
 'ISIC_2019_Training_Metadata.csv',
 'train.csv',
 'расшифровка.txt']

In [29]:
base_dir = 'D:/Анна/универ/ДИПЛОМ/data/ISIC'
# train_dir
train_dir = os.path.join(base_dir, 'train_dir')
os.mkdir(train_dir)

# test_dir
test_dir = os.path.join(base_dir, 'test_dir')
os.mkdir(test_dir)

#val_dir
val_dir = os.path.join(base_dir, 'val_dir')
os.mkdir(val_dir)

In [30]:
# create new folders inside train_dir
nv = os.path.join(train_dir, 'nv')
os.mkdir(nv)
mel = os.path.join(train_dir, 'mel')
os.mkdir(mel)
bkl = os.path.join(train_dir, 'bkl')
os.mkdir(bkl)
bcc = os.path.join(train_dir, 'bcc')
os.mkdir(bcc)
ak = os.path.join(train_dir, 'ak')
os.mkdir(ak)
vasc = os.path.join(train_dir, 'vasc')
os.mkdir(vasc)
df = os.path.join(train_dir, 'df')
os.mkdir(df)
scc=os.path.join(train_dir, 'scc')
os.mkdir(scc)
unk=os.path.join(train_dir, 'unk')
os.mkdir(unk)

In [31]:
# create new folders inside test_dir
nv = os.path.join(test_dir, 'nv')
os.mkdir(nv)
mel = os.path.join(test_dir, 'mel')
os.mkdir(mel)
bkl = os.path.join(test_dir, 'bkl')
os.mkdir(bkl)
bcc = os.path.join(test_dir, 'bcc')
os.mkdir(bcc)
ak = os.path.join(test_dir, 'ak')
os.mkdir(ak)
vasc = os.path.join(test_dir, 'vasc')
os.mkdir(vasc)
df = os.path.join(test_dir, 'df')
os.mkdir(df)
scc=os.path.join(test_dir, 'scc')
os.mkdir(scc)
unk=os.path.join(test_dir, 'unk')
os.mkdir(unk)

In [32]:
# create new folders inside val_dir
nv = os.path.join(val_dir, 'nv')
os.mkdir(nv)
mel = os.path.join(val_dir, 'mel')
os.mkdir(mel)
bkl = os.path.join(val_dir, 'bkl')
os.mkdir(bkl)
bcc = os.path.join(val_dir, 'bcc')
os.mkdir(bcc)
ak = os.path.join(val_dir, 'ak')
os.mkdir(ak)
vasc = os.path.join(val_dir, 'vasc')
os.mkdir(vasc)
df = os.path.join(val_dir, 'df')
os.mkdir(df)
scc=os.path.join(val_dir, 'scc')
os.mkdir(scc)
unk=os.path.join(val_dir, 'unk')
os.mkdir(unk)

In [34]:
df_data = pd.read_csv('D:\Анна\универ\ДИПЛОМ\data\ISIC\ISIC_2019_Training_Metadata.csv')
df_data.head()

,image,age_approx,anatom_site_general,lesion_id,sex
0,ISIC_0000000,55.0,anterior torso,NaN,female
1,ISIC_0000001,30.0,anterior torso,NaN,female
2,ISIC_0000002,60.0,upper extremity,NaN,female
3,ISIC_0000003,30.0,upper extremity,NaN,male
4,ISIC_0000004,80.0,posterior torso,NaN,male


In [35]:
df_got = pd.read_csv("D:\Анна\универ\ДИПЛОМ\data\ISIC\ISIC_2019_Training_GroundTruth.csv")
df_got.head()

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
0,ISIC_0000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
df = pd.concat([df_data, df_got.drop('image',axis=1)], axis=1)
df.head()

,image,age_approx,anatom_site_general,lesion_id,sex,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
0,ISIC_0000000,55.0,anterior torso,NaN,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0000001,30.0,anterior torso,NaN,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0000002,60.0,upper extremity,NaN,female,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0000003,30.0,upper extremity,NaN,male,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0000004,80.0,posterior torso,NaN,male,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
for i in df.index:
    df.loc[i,'lesion_id']=np.where(df.loc[i,'MEL'] > 0, str("mel"),
                                  np.where(df.loc[i,'NV']>0, str("nv"),
                                          np.where(df.loc[i,'BCC']>0, str("bcc"),
                                                  np.where (df.loc[i,'AK']>0, str("ak"),
                                                           np.where (df.loc[i,'BKL']>0, str("bkl"),
                                                                    np.where (df.loc[i,'DF']>0, str("df"),
                                                                             np.where(df.loc[i,'VASC']>0, str("vasc"),
                                                                                     np.where(df.loc[i,'SCC']>0, str("scc"),
                                                                                             np.where(df.loc[i,'UNK']>0, str("unk"), "NaN")))))))))
df.head()

,image,age_approx,anatom_site_general,lesion_id,sex,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
0,ISIC_0000000,55.0,anterior torso,nv,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0000001,30.0,anterior torso,nv,female,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0000002,60.0,upper extremity,mel,female,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0000003,30.0,upper extremity,nv,male,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0000004,80.0,posterior torso,mel,male,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
for i in df.index:
    df.loc[i,'lesion_id']=str(df.loc[i,'lesion_id'])
df.loc[1,'lesion_id']

'nv'

In [71]:
df2=df[['image','lesion_id']]
df2.dropna()

,image,lesion_id
0,ISIC_0000000,nv
1,ISIC_0000001,nv
2,ISIC_0000002,mel
3,ISIC_0000003,nv
4,ISIC_0000004,mel
...,...,...
25326,ISIC_0073247,bcc
25327,ISIC_0073248,bkl
25328,ISIC_0073249,mel
25329,ISIC_0073251,nv


In [72]:
#df2.to_csv(base_dir+'/train.csv')

In [73]:
y = df2['lesion_id']
df_train, df_test = train_test_split(df2, train_size=0.15, test_size=0.015, random_state=101, stratify=y)

In [74]:
y = df_train['lesion_id']
df_train, df_val = train_test_split(df_train, train_size=0.8, test_size=0.1, random_state=101, stratify=y)

In [75]:
df_train['lesion_id'].value_counts()

nv      1545
mel      542
bcc      398
bkl      315
ak       104
scc       75
vasc      31
df        29
Name: lesion_id, dtype: int64

In [76]:
df_val['lesion_id'].value_counts()

nv      193
mel      68
bcc      50
bkl      40
ak       13
scc      10
df        3
vasc      3
Name: lesion_id, dtype: int64

In [77]:
df_test['lesion_id'].value_counts()

nv      193
mel      68
bcc      50
bkl      39
ak       13
scc       9
df        4
vasc      4
Name: lesion_id, dtype: int64

In [78]:
# Set the image_id as the index in df_data
df2.set_index('image', inplace=True)

In [79]:
#def name(row):
  #  row.image = row.image + '.jpg'
 #   return row
#df2=df2.apply(name,axis=1)
#df_train = df_train.apply(name,axis = 1)
#df_val=df_val.apply(name,axis=1)

In [80]:
# Get a list of train and val images
train_list = list(df_train['image'])
val_list = list(df_val['image'])
test_list=list(df_test['image'])

In [81]:
# Transfer 
for image in train_list:
    fname = image +'.jpg'
    mlabel = df2.loc[image,'lesion_id']
    src = os.path.join('D:/Анна/универ/ДИПЛОМ/data/ISIC/ISIC_2019_Training_Input', fname)
        # destination path to image
    dst = os.path.join(train_dir, mlabel, fname)
        # copy the image from the source to the destination
    shutil.copyfile(src, dst)

In [82]:
for image in test_list:
    fname = image +'.jpg'
    mlabel = df2.loc[image,'lesion_id']
    src = os.path.join('D:/Анна/универ/ДИПЛОМ/data/ISIC/ISIC_2019_Training_Input', fname)
        # destination path to image
    dst = os.path.join(test_dir, mlabel, fname)
        # copy the image from the source to the destination
    shutil.copyfile(src, dst)

In [83]:
for image in val_list:
    fname = image +'.jpg'
    mlabel = df2.loc[image,'lesion_id']
    src = os.path.join('D:/Анна/универ/ДИПЛОМ/data/ISIC/ISIC_2019_Training_Input', fname)
        # destination path to image
    dst = os.path.join(val_dir, mlabel, fname)
        # copy the image from the source to the destination
    shutil.copyfile(src, dst)

class_list = ['nv','mel','bkl','bcc','ak','vasc','df', 'scc']
dir_list= [train_dir, val_dir, test_dir]
for maindir in dir_list:
    for item in class_list:

        # We are creating temporary directories here because we delete these directories later
        # create a base dir
        aug_dir = os.path.join(base_dir,'aug_dir')
        os.mkdir(aug_dir)
        # create a dir within the base dir to store images of the same class
        img_dir = os.path.join(aug_dir, 'img_dir')
        os.mkdir(img_dir)

        # Choose a class
        img_class = item

        # list all images in that directory
        img_list = os.listdir(maindir + '/' + img_class)


        # Copy images from the class train dir to the img_dir 
        for fname in img_list:
                src = os.path.join(maindir , img_class, fname)
                dst = os.path.join(img_dir, fname)
                shutil.copyfile(src, dst)


        # point to a dir containing the images and not to the images themselves
        path = aug_dir
        save_path = maindir + '/' + img_class

        # Create a data generator
        datagen = ImageDataGenerator(rescale=1./255)

        batch_size = 50

        aug_datagen = datagen.flow_from_directory(path,
                                               save_to_dir=save_path,
                                               save_format='jpg',
                                                        target_size=(224,224),
                                                        batch_size=batch_size)

        # delete temporary directory with the raw image files
        shutil.rmtree(aug_dir)